In [2]:
pip install folium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/90/37/08e416c9915dcf7d53deb0fbdb702266902c584617dfa6e6c84fb2fc6ee3/geopandas-0.14.3-py3-none-any.whl.metadata
  Obtaining dependency information for fiona>=1.8.21 from https://files.pythonhosted.org/packages/7f/27/b24c1610c7ae5716709321f04d38d7b8b71ed531f80df4f697b9ad99cfc3/fiona-1.9.5-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/51.1 kB ? eta -:--:--
     -------------------------------------- 51.1/51.1 kB 869.6 kB/s eta 0:00:00
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/79/95/eb68113c5b5737c342bde1bab92705dabe69c16299c5a122616e50f1fbd6/pyproj-3.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for shapely>=1.8.0 from https://files.py

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
import pandas as pd
import numpy as np
import os
import requests
import pprint
import json
import matplotlib.pyplot as plt
import pandas as pd
import folium


In [5]:
state_geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'
geo = state_geo

In [6]:
# params ---- 1.
center = [37.541, 126.986]
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']

# visualization ---- 2.
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


In [7]:
folium.GeoJson(
    geo,
    name = '서초구',
    style_function = lambda x: {'fillColor':'#00000000' ,
    							'color':'black',
                                'weight' :'1'}
).add_to(m)
m

In [14]:
file_path = './장애인+현황(장애유형별).xlsx'

# 엑셀 파일 불러오기
df = pd.read_excel(file_path)

# '계'에 해당하는 열 인덱스 찾기 (첫 번째 '2022')
col_index = df.columns.get_loc("2022")  # '계'에 해당하는 첫 번째 '2022' 열 인덱스

# 데이터 선택 (첫 3행을 제외하고 '소계'가 아닌 모든 행 선택)
filtered_df = df.iloc[3:, [1, col_index]]

# 새로운 데이터프레임 생성
new_df = filtered_df.rename(columns={'자치구별(2)': '자치구별', df.columns[col_index]: '계'})

no_totals_df = new_df[new_df['자치구별'] != '소계']

admin_division_file_path = './korea_administrative_division_latitude_longitude.xlsx'
admin_div_df = pd.read_excel(admin_division_file_path)

# 서울시 자치구만 필터링
seoul_admin_div_df = admin_div_df[admin_div_df['do'] == '서울']

# 'city' 열을 기준으로 no_totals_df와 결합
# no_totals_df의 '자치구별' 열과 seoul_admin_div_df의 'city' 열이 일치해야 합니다.
# '자치구별' 열의 이름을 'city'로 변경하여 일치시킴
no_totals_df_renamed = no_totals_df.rename(columns={'자치구별': 'city'})

# Merge (결합) 작업
merged_df = pd.merge(seoul_admin_div_df, no_totals_df_renamed, on='city')

In [15]:
folium.GeoJson(
    geo,
    name = '서초구',
    style_function = lambda x: {'fillColor':'#00000000' ,
    							'color':'black',
                                'weight' :'1'}
).add_to(m)



import folium

# 서울시 중심의 기본 지도 생성
seoul_map = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 각 자치구별로 원형 마커 추가
for idx, row in merged_df.iterrows():
    
    radius = row['계'] / 1000 
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=radius, # 반지름
                        color='blue',
                        fill=True,
                        fill_color='blue',
                        fill_opacity=0.6,
                        popup=f"{row['city']}: {row['계']}").add_to(seoul_map)



In [20]:
import folium
import pandas as pd

# 서울시 중심의 기본 지도 생성 및 타일 옵션
tiles = 'cartodbpositron'
seoul_map = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# GeoJSON 데이터 로드 (여기서는 'geo'를 예시 데이터로 가정)
geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

# GeoJSON 레이어 추가
folium.GeoJson(
    geo,
    name='서울시 자치구',
    style_function=lambda x: {'fillColor': '#00000000', 'color': 'black', 'weight': '4'}
).add_to(seoul_map)

# DataFrame 로드 (예: 'merged_df')
# merged_df = pd.read_csv('path_to_your_dataframe.csv')

# 각 자치구별 원형 마커 추가
for idx, row in merged_df.iterrows():
    radius = row['계'] / 1000  # 인구를 기준으로 반지름 설정
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,
        color='black',
        fill=True,
        fill_color='white',
        fill_opacity=0.6,
        popup=f"{row['city']}: {row['계']}"
    ).add_to(seoul_map)

seoul_map